In [7]:
! pip install gym-anytrading
! pip install gym numpy stable-baselines3


     |████████████████████████████████| 721 kB 651 kB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 183 kB 10.5 MB/s eta 0:00:01
     |████████████████████████████████| 766.7 MB 3.9 kB/s  eta 0:00:01    |██▌                             | 60.4 MB 5.9 MB/s eta 0:02:01     |███████████████████████████████▎| 749.6 MB 8.3 MB/s eta 0:00:03
     |████████████████████████████████| 6.2 MB 28.5 MB/s eta 0:00:01
     |████████████████████████████████| 883 kB 104.8 MB/s eta 0:00:01
     |████████████████████████████████| 150.1 MB 10 kB/s  eta 0:00:011
     |████████████████████████████████| 207.5 MB 32 kB/s s eta 0:00:01
     |████████████████████████████████| 21.1 MB 15.4 MB/s eta 0:00:01
     |████████████████████████████████| 664.8 MB 21 kB/s s eta 0:00:01    |██▊                             | 55.6 MB 49.7 MB/s eta 0:00:13     |█████████▎                      | 1

In [2]:
import gymnasium as gym
import gym_anytrading

env = gym.make('forex-v0')
# env = gym.make('stocks-v0')

In [8]:
import numpy as np
import matplotlib.pyplot as plt

import gymnasium as gym
import gym_anytrading
from gym_anytrading.envs import TradingEnv, ForexEnv, StocksEnv, Actions, Positions 
from gym_anytrading.datasets import FOREX_EURUSD_1H_ASK, STOCKS_GOOGL
from stable_baselines3 import DQN  # or PPO, A2C, etc.


# env = gym.make('forex-v0', frame_bound=(50, 100), window_size=10)
env = gym.make('stocks-v0', frame_bound=(50, 100), window_size=10)

observation = env.reset(seed=2023)
while True:
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated

    # env.render()
    if done:
        print("info:", info)
        break

plt.cla()
env.unwrapped.render_all()
plt.show()

ModuleNotFoundError: No module named 'stable_baselines3'

In [5]:
model = DQN("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10_000)


NameError: name 'DQN' is not defined